# Fuzzywuzzy

**Inhalt:** Wie ähnlich sind zwei Strings? II

**Nötige Skills:** keine

**Lernziele:** Eine andere Möglichkeit für fuzzy string matching kennenlernen

## About

Fuzzywuzzy
- Package https://github.com/seatgeek/fuzzywuzzy
- Dokumentation: fast nicht vorhanden

Installation:

```bash
pip3 install fuzzywuzzy[speedup]
```


## Setup

In [1]:
import pandas as pd

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Basics

Die Fuzzywuzzy-Funktionen spucken eine Zahl zwischen 0 und 100 aus, welche die Ähnlichkeit angibt.

In [3]:
fuzz.ratio("a", "b")

0

In [4]:
fuzz.ratio("a", "a")

100

## Scorer-Funktionen

### ratio

Jedes einzelne Zeichen ist wichtig, auch die Reihenfolge.

In [5]:
fuzz.ratio("Peter Pan", "Peter Pan")

100

In [6]:
fuzz.ratio("Peter Pan", "Dr. Peter Pan")

82

### partial_ratio

Partielle Übereinstimmung der Strings ist ok. Allerdings muss die Reihenfolge der Zeichen stimmen:

In [7]:
fuzz.partial_ratio("Peter Pan", "Dr. Peter Pan")

100

In [8]:
fuzz.partial_ratio("Peter Pan", "Peter V. Pan")

67

In [9]:
fuzz.partial_ratio("Peter Pan", "Pan, Peter")

56

### token_sort_ratio

Sortiert die Wörter zuerst. Gibt aber Abzüge für partielle Matches:

In [10]:
fuzz.token_sort_ratio("Peter Pan", "Pan, Peter")

100

In [11]:
fuzz.token_sort_ratio("Peter Pan", "Peter V. Pan")

90

### token_set_ratio

Kommt klar mit vertauschter Reihenfolge und partiellen Matches:

In [12]:
fuzz.token_set_ratio("Peter Pan", "Peter V. Pan")

100

In [13]:
fuzz.token_set_ratio("Peter Pan", "Pan, Peter Dr.")

100

Abzüge gibt es für falsche Zeichen innerhalb der Wörter:

In [14]:
fuzz.token_set_ratio("Peter Pan", "Pann, Peter Dr.")

82

Der Score geht recht rapide runter. Zum Vergleich:

In [15]:
fuzz.token_set_ratio("Peter Pan", "Paul Pan")

59

## Beispiel in Pandas

Wir wollen die Daten in einer Tabelle "putzen". Das heisst: die verschiedenen Schreibweisen von Begriffen vereinheitlichen (z.B. "Pan, Peter" - "Peter Pan")

In [16]:
df = pd.read_csv('dataprojects/Jellyfish/Words.csv')

In [17]:
df

,Wort 1,Wort 2
0,Tisch,Tische
1,Tisch,Fisch
2,Tisch,Dorf
3,Peter Müller,Peter Mueller
4,Peter Müller,Pete Müller
5,Peter Müller,Pete Miller
6,Peter Pan,Peter Pan
7,Peter Pan,Peter V. Pan
8,Peter Pan,P. Pan
9,Peter Pan,"Pan, Peter"


### Score testen

Wir können mit `df.apply()` ähnlich wie vorher den Score eines Vergleichs testen und in einer neuen Spalte festhalten,

In [18]:
df['token_set_ratio'] = df.apply(lambda row: fuzz.token_set_ratio(row['Wort 1'], row['Wort 2']) ,axis=1)

In [19]:
df

,Wort 1,Wort 2,token_set_ratio
0,Tisch,Tische,91
1,Tisch,Fisch,80
2,Tisch,Dorf,0
3,Peter Müller,Peter Mueller,92
4,Peter Müller,Pete Müller,95
5,Peter Müller,Pete Miller,91
6,Peter Pan,Peter Pan,100
7,Peter Pan,Peter V. Pan,100
8,Peter Pan,P. Pan,75
9,Peter Pan,"Pan, Peter",100


In [20]:
df['match'] = df['token_set_ratio'] > 80

In [21]:
df

,Wort 1,Wort 2,token_set_ratio,match
0,Tisch,Tische,91,True
1,Tisch,Fisch,80,False
2,Tisch,Dorf,0,False
3,Peter Müller,Peter Mueller,92,True
4,Peter Müller,Pete Müller,95,True
5,Peter Müller,Pete Miller,91,True
6,Peter Pan,Peter Pan,100,True
7,Peter Pan,Peter V. Pan,100,True
8,Peter Pan,P. Pan,75,False
9,Peter Pan,"Pan, Peter",100,True


**Frage:** Wie steht es hier um die false positives / false negatives?

In [22]:
# false positives:
# 

In [23]:
# false negatives:
# 

### Wörter ersetzen

Was uns aber eigentlich interessiert: die neue Liste.

**a) Alle Varianten eines Begriffs ersetzen**

Die Funktion `process()` ist dafür hilfreich

Sortiert eine Liste/Dictionary entsprechend dem besten Match...
- Welchen String wollen wir vergleichen? "Peter Pan"
- Welche Liste wollen wir durchsuchen? "Wort 2"
- Welchen Scorer wollen wir verwenden? token_set_ratio
- Wie viele Ergebnisse sollen angezeigt werden? 20

...und retourniert die Liste mit dem Matching-Score:

In [24]:
match_list = process.extract("Peter Pan", df['Wort 2'], scorer=fuzz.token_set_ratio, limit=20)

Wir vergleichen also jede einzelne Zeile in der Tabelle mit dem String "Peter Pan":

In [25]:
match_list

[('Peter Pan', 100, 6),
 ('Peter V. Pan', 100, 7),
 ('Pan, Peter', 100, 9),
 ('Dr. Peter Pan', 100, 10),
 ('P. Pan', 75, 8),
 ('Peter Mueller', 71, 3),
 ('Pete Müller', 53, 4),
 ('Pete Miller', 50, 5),
 ('Novartis Health AG', 37, 13),
 ('Novartis AG', 30, 11),
 ('Tische', 27, 0),
 ('Novartis Schweiz', 24, 12),
 ('Dorf', 15, 2),
 ('Fisch', 0, 1)]

Für unsere Auswahl brauchen wir aber einen bestimmten "Cut-Off-Punkt".

Zum Beispiel können wir sagen, dass wir nur die Schreibweisen ersetzen wollen, die einen Score über 80 erzielen...

... wir speichern diese Varianten in einer Liste.

In [26]:
variationen = [match[0] for match in match_list if match[1] >= 80]
variationen

['Peter Pan', 'Peter V. Pan', 'Pan, Peter', 'Dr. Peter Pan']

Anhand dieser Liste können wir nun selektiv die Schreibweise in einzelnen Zeilen ändern.

In [27]:
df.loc[df['Wort 2'].isin(variationen), 'New Wort 2'] = "Peter Pan"

In [28]:
df

,Wort 1,Wort 2,token_set_ratio,match,New Wort 2
0,Tisch,Tische,91,True,NaN
1,Tisch,Fisch,80,False,NaN
2,Tisch,Dorf,0,False,NaN
3,Peter Müller,Peter Mueller,92,True,NaN
4,Peter Müller,Pete Müller,95,True,NaN
5,Peter Müller,Pete Miller,91,True,NaN
6,Peter Pan,Peter Pan,100,True,Peter Pan
7,Peter Pan,Peter V. Pan,100,True,Peter Pan
8,Peter Pan,P. Pan,75,False,NaN
9,Peter Pan,"Pan, Peter",100,True,Peter Pan


**b) Ersetzen anhand einer bekannten Liste von richtigen Begriffen**

Vorausgesetzt, wir kennen die Liste der tatsächlichen Begriffe, können wir auch gleich alle auf einmal handhaben:

In [29]:
correct_terms = [
    'Tisch',
    'Peter Müller',
    'Peter Pan',
    'Novartis'
]

Die Funktion `process.extractOne()` funktioniert wie `process.extract()`, aber gibt nur den besten Match zurück:

In [30]:
process.extractOne("Dorf", correct_terms, scorer=fuzz.token_set_ratio)

('Novartis', 33)

Wir können uns eine "Tester"-Funktion basteln:
- Findet den besten Match aus der liste der korrekten Strings
- Falls kein Match gut genug ist, wird das Wort nicht ersetzt
- Falls der Match gut genug ist, wir das Wort ersetzt

In [31]:
def best_match(word):
    match = process.extractOne(word, correct_terms, scorer=fuzz.token_set_ratio)
    if match[1] < 80:
        best_match = word
    else:
        best_match = match[0]
    return best_match

Angewendet auf die Tabelle sieht das dann so aus:

In [32]:
df['Wort 2'].apply(best_match)

0            Tisch
1            Tisch
2             Dorf
3     Peter Müller
4     Peter Müller
5     Peter Müller
6        Peter Pan
7        Peter Pan
8           P. Pan
9        Peter Pan
10       Peter Pan
11        Novartis
12        Novartis
13        Novartis
Name: Wort 2, dtype: object

Speichern in einer neuen Spalte:

In [33]:
df['New Wort 2'] = df['Wort 2'].apply(best_match)
df

,Wort 1,Wort 2,token_set_ratio,match,New Wort 2
0,Tisch,Tische,91,True,Tisch
1,Tisch,Fisch,80,False,Tisch
2,Tisch,Dorf,0,False,Dorf
3,Peter Müller,Peter Mueller,92,True,Peter Müller
4,Peter Müller,Pete Müller,95,True,Peter Müller
5,Peter Müller,Pete Miller,91,True,Peter Müller
6,Peter Pan,Peter Pan,100,True,Peter Pan
7,Peter Pan,Peter V. Pan,100,True,Peter Pan
8,Peter Pan,P. Pan,75,False,P. Pan
9,Peter Pan,"Pan, Peter",100,True,Peter Pan


**Aber:** Was ist, wenn wir die Liste der "richtigen" Schreibweisen gar nicht besitzen???

In [34]:
# 